In [10]:
import requests
import pandas as pd

In [11]:
series_ids = {
    
'CPI':'CUUR0000SA0', 
'Unemployment':'LNS14000000', 
'NF Employment':'CES0000000001', 
'ULC':'PRS85006112', 
'Hourly earnings':'CES0500000003', 
'PPI':'WPUFD4', 
'Shelter':'CUSR0000SAH1', 
'Energy':'CUUR0000SA0E', 
'Medical Care': 'CUSR0000SAM',
'Food':'CUUR0000SAF1',
'Gasoline': 'APU000074714',
'Imports': 'EIUIR',
'New & Used Vehicles': 'CUSR0000SETA',
'All Items Less Food & Shelter': 'CUSR0000SA0L12E'
  

}



In [12]:
api_key = '178d3446c4fe4ee388252bdeb9bb243e'  

def create_df(id):
    series_id = id  
    start_year = '2013'
    end_year = '2023'

    url = f'https://api.bls.gov/publicAPI/v2/timeseries/data/'

    data = {
        'seriesid': [series_id],
        'startyear': start_year,
        'endyear': end_year,
        'registrationKey': api_key
    }

    response = requests.post(url, json=data)
    data = response.json()

    # Convert to DataFrame
    df = pd.DataFrame(data['Results']['series'][0]['data'])
    df['value'] = df['value'].astype(float)
    df['yoy'] = 100*((df['value']/df['value'].shift(-12))-1)
    df = df[['year', 'periodName', 'value', 'yoy']]

    return(df)
    

In [13]:

combined_df = pd.DataFrame()


for series_name, series_id in series_ids.items():

    df = create_df(series_id)
    
    df = df.rename(columns={'value': series_name, 'yoy': f'{series_name}_yoy'})

    if combined_df.empty:
        combined_df = df
    else:
        combined_df = pd.merge(combined_df, df, on=['year', 'periodName'], how='outer')

print(combined_df.head())

   year periodName      CPI   CPI_yoy  Unemployment  Unemployment_yoy  \
0  2023   December  306.746  3.352123           3.7          5.714286   
1  2023   November  307.051  3.137271           3.7          2.777778   
2  2023    October  307.671  3.241145           3.8          5.555556   
3  2023  September  307.789  3.699698           3.8          8.571429   
4  2023     August  307.026  3.665112           3.8          5.555556   

   NF Employment  NF Employment_yoy  ULC  ULC_yoy  ...     Food  Food_yoy  \
0       157347.0           1.980673  NaN      NaN  ...  325.409  2.704844   
1       157014.0           1.854627  NaN      NaN  ...  325.172  2.949119   
2       156832.0           1.907120  NaN      NaN  ...  325.731  3.300742   
3       156667.0           2.039261  NaN      NaN  ...  324.704  3.692255   
4       156421.0           2.048525  NaN      NaN  ...  324.100  4.254121   

   Gasoline  Gasoline_yoy  Imports  Imports_yoy  New & Used Vehicles  \
0     3.289     -1.996424 

In [14]:
combined_df.head()

,year,periodName,CPI,CPI_yoy,Unemployment,Unemployment_yoy,NF Employment,NF Employment_yoy,ULC,ULC_yoy,...,Food,Food_yoy,Gasoline,Gasoline_yoy,Imports,Imports_yoy,New & Used Vehicles,New & Used Vehicles_yoy,All Items Less Food & Shelter,All Items Less Food & Shelter_yoy
0,2023,December,306.746,3.352123,3.7,5.714286,157347.0,1.980673,NaN,NaN,...,325.409,2.704844,3.289,-1.996424,139.5,-1.622003,127.596,0.080004,270.725,2.184670
1,2023,November,307.051,3.137271,3.7,2.777778,157014.0,1.854627,NaN,NaN,...,325.172,2.949119,3.500,-9.161692,139.5,-1.483051,127.908,-0.096070,270.202,2.107125
2,2023,October,307.671,3.241145,3.8,5.555556,156832.0,1.907120,NaN,NaN,...,325.731,3.300742,3.782,-5.826693,140.2,-1.751927,126.772,-1.819984,269.779,2.020913
3,2023,September,307.789,3.699698,3.8,8.571429,156667.0,2.039261,NaN,NaN,...,324.704,3.692255,3.988,2.757021,141.0,-1.467505,127.343,-1.915582,269.390,1.930855
4,2023,August,307.026,3.665112,3.8,5.555556,156421.0,2.048525,NaN,NaN,...,324.100,4.254121,3.955,-3.560107,140.3,-2.973721,128.323,-0.803172,269.199,2.318519


In [15]:
combined_df.drop(columns='Unemployment_yoy', inplace=True)
combined_df.head()

,year,periodName,CPI,CPI_yoy,Unemployment,NF Employment,NF Employment_yoy,ULC,ULC_yoy,Hourly earnings,...,Food,Food_yoy,Gasoline,Gasoline_yoy,Imports,Imports_yoy,New & Used Vehicles,New & Used Vehicles_yoy,All Items Less Food & Shelter,All Items Less Food & Shelter_yoy
0,2023,December,306.746,3.352123,3.7,157347.0,1.980673,NaN,NaN,34.36,...,325.409,2.704844,3.289,-1.996424,139.5,-1.622003,127.596,0.080004,270.725,2.184670
1,2023,November,307.051,3.137271,3.7,157014.0,1.854627,NaN,NaN,34.23,...,325.172,2.949119,3.500,-9.161692,139.5,-1.483051,127.908,-0.096070,270.202,2.107125
2,2023,October,307.671,3.241145,3.8,156832.0,1.907120,NaN,NaN,34.10,...,325.731,3.300742,3.782,-5.826693,140.2,-1.751927,126.772,-1.819984,269.779,2.020913
3,2023,September,307.789,3.699698,3.8,156667.0,2.039261,NaN,NaN,34.01,...,324.704,3.692255,3.988,2.757021,141.0,-1.467505,127.343,-1.915582,269.390,1.930855
4,2023,August,307.026,3.665112,3.8,156421.0,2.048525,NaN,NaN,33.91,...,324.100,4.254121,3.955,-3.560107,140.3,-2.973721,128.323,-0.803172,269.199,2.318519


In [16]:
combined_df.tail()

,year,periodName,CPI,CPI_yoy,Unemployment,NF Employment,NF Employment_yoy,ULC,ULC_yoy,Hourly earnings,...,Food,Food_yoy,Gasoline,Gasoline_yoy,Imports,Imports_yoy,New & Used Vehicles,New & Used Vehicles_yoy,All Items Less Food & Shelter,All Items Less Food & Shelter_yoy
171,2014,1st Quarter,NaN,NaN,NaN,NaN,NaN,12.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2013,4th Quarter,NaN,NaN,NaN,NaN,NaN,-0.8,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,2013,3rd Quarter,NaN,NaN,NaN,NaN,NaN,-3.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,2013,2nd Quarter,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,2013,1st Quarter,NaN,NaN,NaN,NaN,NaN,-8.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 29 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   year                               176 non-null    object 
 1   periodName                         176 non-null    object 
 2   CPI                                132 non-null    float64
 3   CPI_yoy                            120 non-null    float64
 4   Unemployment                       132 non-null    float64
 5   NF Employment                      132 non-null    float64
 6   NF Employment_yoy                  120 non-null    float64
 7   ULC                                44 non-null     float64
 8   ULC_yoy                            32 non-null     float64
 9   Hourly earnings                    132 non-null    float64
 10  Hourly earnings_yoy                120 non-null    float64
 11  PPI                                132 non-null    float64

In [18]:
combined_df.to_csv('BLS_data.csv')

In [19]:
# for individual pulls
'''
series_id = id  
start_year = '2013'
end_year = '2023'

url = f'https://api.bls.gov/publicAPI/v2/timeseries/data/'

data = {
    'seriesid': 'CES0000000001',
    'startyear': start_year,
    'endyear': end_year,
    'registrationKey': api_key
}

response = requests.post(url, json=data)
data = response.json()

# Convert to DataFrame
df1 = pd.DataFrame(data['Results']['series'][0]['data'])
df1['value'] = df1['value'].astype(float)
df1['yoy'] = 100*((df1['value']/df1['value'].shift(-12))-1)
df1 = df1[['year', 'periodName', 'value', 'yoy']]

return(df1)
'''

"\nseries_id = id  \nstart_year = '2013'\nend_year = '2023'\n\nurl = f'https://api.bls.gov/publicAPI/v2/timeseries/data/'\n\ndata = {\n    'seriesid': 'CES0000000001',\n    'startyear': start_year,\n    'endyear': end_year,\n    'registrationKey': api_key\n}\n\nresponse = requests.post(url, json=data)\ndata = response.json()\n\n# Convert to DataFrame\ndf1 = pd.DataFrame(data['Results']['series'][0]['data'])\ndf1['value'] = df1['value'].astype(float)\ndf1['yoy'] = 100*((df1['value']/df1['value'].shift(-12))-1)\ndf1 = df1[['year', 'periodName', 'value', 'yoy']]\n\nreturn(df1)\n"

In [7]:
#df2 = pd.read_csv('first_data.csv')

In [8]:
#df2.head()

,Unnamed: 0,year,periodName,CPI,CPI_yoy,Unemployment,NF Employment,NF Employment_yoy,ULC,ULC_yoy,Hourly earnings,Hourly earnings_yoy,PPI,PPI_yoy,Shelter,Shelter_yoy,Energy,Energy_yoy,Food,Food_yoy
0,0,2023,December,306.746,3.352123,3.7,157232.0,1.745236,NaN,NaN,34.27,4.100851,141.154,0.982973,390.037,6.174409,269.375,-2.023009,325.409,2.704844
1,1,2023,November,307.051,3.137271,3.7,157016.0,1.762845,NaN,NaN,34.12,4.024390,141.877,0.775651,388.266,6.526010,277.029,-5.435684,325.172,2.949119
2,2,2023,October,307.671,3.241145,3.8,156843.0,1.842136,NaN,NaN,34.00,4.102878,142.448,1.213585,386.536,6.730727,286.754,-4.529580,325.731,3.300742
3,3,2023,September,307.789,3.699698,3.8,156738.0,1.988522,NaN,NaN,33.91,4.242238,142.898,2.021904,385.259,7.145857,296.004,-0.450322,324.704,3.692255
4,4,2023,August,307.026,3.665112,3.8,156476.0,2.050453,NaN,NaN,33.82,4.286155,142.641,1.911907,382.782,7.248886,294.328,-3.616573,324.100,4.254121


New tickers

This is good for inflation (one screen in data finder) https://data.bls.gov/PDQWeb/cu and can get by region for CPI stuff


Mortgage interest : CXUOWNMORTGLB0401M (only annual, latest 2022)
https://beta.bls.gov/dataViewer/view/timeseries/CXUOWNMORTGLB0401M

Gasoline : APU000074714

Imports (all commodities): EIUIR

Consumer exp survey https://data.bls.gov/cgi-bin/surveymost?cx :

Food: CXUFOODTOTLLB0201M (this is annual)

Healthcare: CXUHEALTHLB0401M (annual)
Medical care: CUSR0000SAM

New and used motor vehicles: 	CUSR0000SETA

All items less food and shelter: CUSR0000SA0L12E

Tax = Income - disposable income - see BEA

Also can see 


BEA: - need to pull this monthly
Much of the below is in there
https://www.bea.gov/sites/default/files/2024-01/pi1223.pdf

Personal savings rate
Personal Interest Income
Govt Social Benefits
Also calculates personal outlays and net is personal saving rate



